### Step 1: load all required packages

In [ ]:
%matplotlib inline

import ast
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.dates import YearLocator
from matplotlib.ticker import ScalarFormatter
from matplotlib.patches import Patch
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Arial'

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as ticker
import matplotlib
import seaborn as sns
from itertools import groupby
from scipy import integrate
from tqdm import tqdm

plt.style.use('ggplot')
pd.set_option('chained_assignment',None)

sys.path.append(os.path.join( '..'))
from miriam_py.utils import load_config
data_path = load_config()['paths']['data']
figure_path = load_config()['paths']['figures']

### Step 2: Create functions required to make the figures

In [ ]:
def sum_tuples(l):
    return tuple(sum(x) for x in zip(*l))

def calc_risk_total(x,hazard,RPS,events):
    collect_risks = []
    for y in range(7):
        collect_risks.append(integrate.simps([x[y] for x in x[events]][::-1], x=RPS[::-1]))
    return collect_risks

def set_prot_standard(x,prot_lookup,events):
    prot_stand = prot_lookup[x.region]
    no_floods= [z for z in events if prot_stand > int(z.split('-')[1])]
    for no_flood in no_floods:
        x[no_flood] = (0,0,0,0,0,0,0)
    return x

def pluvial_design(x,hazard):
    if x.GroupCode == 'HIC':
        if x.road_type in ['primary','secondary']:
            if hazard == 'PU':
                x[['PU-5','PU-10','PU-20','PU-50']] = [(0,0,0,0,0,0,0)]*4
            elif hazard == 'FU':
                x[['FU-5','FU-10','FU-20','FU-50']] = [(0,0,0,0,0,0,0)]*4
            elif hazard == 'CF':
                x[['CF-10','CF-20','CF-50']] = [(0,0,0,0,0,0,0)]*3               
        else:
            if hazard == 'PU':
                x[['PU-5','PU-10','PU-20']] = [(0,0,0,0,0,0,0)]*3
            elif hazard == 'FU':
                x[['FU-5','FU-10','FU-20']] = [(0,0,0,0,0,0,0)]*3
            elif hazard == 'CF':
                x[['CF-10','CF-20']] = [(0,0,0,0,0,0,0)]*2
    elif x.GroupCode == 'UMC':
        if x.road_type in ['primary','secondary']:
            if hazard == 'PU':
                x[['PU-5','PU-10','PU-20']] = [(0,0,0,0,0,0,0)]*3
            elif hazard == 'FU':
                x[['FU-5','FU-10','FU-20']] = [(0,0,0,0,0,0,0)]*3
            elif hazard == 'CF':
                x[['CF-10','CF-20']] = [(0,0,0,0,0,0,0)]*2
        else:
            if hazard == 'PU':
                x[['PU-5','PU-10']] = [(0,0,0,0,0,0,0)]*2
            elif hazard == 'FU':
                x[['FU-5','FU-10']] = [(0,0,0,0,0,0,0)]*2
            elif hazard == 'CF':
                x[['CF-10']] = [(0,0,0,0,0,0,0)]*1
    else:
        if x.road_type in ['primary','secondary']:
            if hazard == 'PU':
                x[['PU-5','PU-10']] = [(0,0,0,0,0,0,0)]*2
            elif hazard == 'FU':
                x[['FU-5','FU-10']] = [(0,0,0,0,0,0,0)]*2
            elif hazard == 'CF':
                x[['CF-10']] = [(0,0,0,0,0,0,0)]*1
       
    return x

def pluvial_design_rail(x,hazard):
    if (hazard == 'PU'):
        if  x.GroupCode == 'HIC':
            x[['PU-5','PU-10','PU-20','PU-50']] = [(0,0,0,0,0,0,0)]*4
        elif x.GroupCode == 'UMC':
            x[['PU-5','PU-10','PU-20','PU-50']] = [(0,0,0,0,0,0,0)]*4
        else:
            x[['PU-5','PU-10','PU-20']] =[(0,0,0,0,0,0,0)]*3

    if (hazard == 'FU'):
        if  x.GroupCode == 'HIC':
            x[['FU-5','FU-10','FU-20','FU-50']] = [(0,0,0,0,0,0,0)]*4
        elif x.GroupCode == 'UMC':
            x[['FU-5','FU-10','FU-20','FU-50']] = [(0,0,0,0,0,0,0)]*4
        else:
            x[['FU-5','FU-10','FU-20']] = [(0,0,0,0,0,0,0)]*3


    if (hazard == 'CF'):
        if  x.GroupCode == 'HIC':
            x[['CF-10','CF-20','CF-50',]] =[(0,0,0,0,0,0,0)]*3
        elif x.GroupCode == 'UMC':
            x[['CF-10','CF-20','CF-50']] = [(0,0,0,0,0,0,0)]*3
        else:
            x[['CF-10','CF-20']] = [(0,0,0,0,0,0,0)]*2

    return x

def get_value(x,ne_sindex,ne_countries,col):
    matches = ne_countries.loc[ne_sindex.intersection(x.centroid.bounds[:2])]
    
    for match in matches.iterrows():
        if match[1].geometry.intersects(x) == True:
            return match[1][col]
        
def gdp_lookup(x):
    try:
        return GDP_lookup[x]
    except:
        return 0

### Step 3: Load datasets with additional information about countries and regions to create the maps

In [ ]:
%%time
global_countries = gpd.read_file(os.path.join(data_path,'input_data','global_countries.shp'))
global_regions = gpd.read_file(os.path.join(data_path,'input_data','global_regions_v2.shp'))
prot_lookup = dict(zip(global_regions['GID_2'],global_regions['prot_stand']))
ne_countries = gpd.read_file(os.path.join(data_path,'input_data','ne_50m_admin_0_countries.shp'))
ne_sindex = ne_countries.sindex

incomegroups = pd.read_csv(os.path.join(data_path,'input_data','incomegroups_2018.csv'),index_col=[0])
global_countries['GDP'] = global_countries.geometry.apply(lambda x: get_value(x,ne_sindex,ne_countries,'GDP_MD_EST'))
global_countries['POP'] = global_countries.geometry.apply(lambda x: get_value(x,ne_sindex,ne_countries,'POP_EST'))



### Step 4: Specify which assets we would like to remove from railway dataset

In [ ]:
rail_to_remove =['disused','abandoned','dismantled','preserved', 'proposed','razed', 
                 'planned','no','historical','na','not_built','abandonned', 'uncompleted', 'demolished',
                 'abandoned_tram','construction;rail', 'rail;construction','waste_disposal','collapsed']

### Step 5: Load all damage estimates

In [ ]:
%%time
events_FU = ['FU-5', 'FU-10', 'FU-20', 'FU-50', 'FU-75', 'FU-100', 'FU-200', 'FU-250','FU-500', 'FU-1000']
tot_road_FU = pd.read_csv(os.path.join(data_path,'summarized','FU_road_losses.csv'),
                          converters = dict(zip(events_FU,[ast.literal_eval]*len(events_FU))),index_col=[0]) 
tot_rail_FU = pd.read_csv(os.path.join(data_path,'summarized','FU_rail_losses.csv'),
                          converters = dict(zip(events_FU,[ast.literal_eval]*len(events_FU))),index_col=[0]) 
print('FU loaded')

events_PU = ['PU-5', 'PU-10', 'PU-20', 'PU-50', 'PU-75', 'PU-100', 'PU-200', 'PU-250', 'PU-500', 'PU-1000']
tot_road_PU = pd.read_csv(os.path.join(data_path,'summarized','PU_road_losses.csv'),
                          converters = dict(zip(events_PU,[ast.literal_eval]*len(events_PU))),index_col=[0]) 
tot_rail_PU = pd.read_csv(os.path.join(data_path,'summarized','PU_rail_losses.csv'),
                          converters = dict(zip(events_PU,[ast.literal_eval]*len(events_PU))),index_col=[0]) 
print('PU loaded')

events_EQ = ['EQ_rp250','EQ_rp475','EQ_rp975','EQ_rp1500','EQ_rp2475'] 
tot_road_EQ = pd.read_csv(os.path.join(data_path,'summarized','EQ_road_losses.csv'),
                          converters = dict(zip(events_EQ,[ast.literal_eval]*len(events_EQ))),index_col=[0]) 
tot_rail_EQ = pd.read_csv(os.path.join(data_path,'summarized','EQ_rail_losses.csv'),
                          converters = dict(zip(events_EQ,[ast.literal_eval]*len(events_EQ))),index_col=[0]) 
print('EQ loaded')

events_Cyc = ['Cyc_rp50','Cyc_rp100','Cyc_rp250','Cyc_rp500','Cyc_rp1000']
tot_road_Cyc = pd.read_csv(os.path.join(data_path,'summarized','Cyc_road_losses_uncer.csv'),
                           converters = dict(zip(events_Cyc,[ast.literal_eval]*len(events_Cyc))))
tot_rail_Cyc = pd.read_csv(os.path.join(data_path,'summarized','Cyc_rail_losses.csv'),
                           converters = dict(zip(events_Cyc,[ast.literal_eval]*len(events_Cyc))))
print('Cyc loaded')

events_CF = ['CF-10', 'CF-20', 'CF-50', 'CF-100', 'CF-200', 'CF-500', 'CF-1000']
tot_road_CF = pd.read_csv(os.path.join(data_path,'summarized','CF_road_losses.csv'),
                          converters = dict(zip(events_CF,[ast.literal_eval]*len(events_CF))),index_col=[0])
tot_rail_CF = pd.read_csv(os.path.join(data_path,'summarized','CF_rail_losses.csv'),
                          converters = dict(zip(events_CF,[ast.literal_eval]*len(events_CF))),index_col=[0])

print('CF loaded')

In [ ]:
%%time
tot_bridge_road = pd.read_csv(os.path.join(data_path,'summarized','bridge_road_risk_.csv'),index_col=[0],
                              converters = dict(zip(['CF_risk','Cyc_risk','EQ_risk','FU_risk','PU_risk'],[ast.literal_eval]*5)))

tot_bridge_road.IncomeGroup = tot_bridge_road.IncomeGroup.apply(lambda x :x.upper())
tot_bridge_rail = pd.read_csv(os.path.join(data_path,'summarized','bridge_rail_risk_.csv'),
                          converters = dict(zip(['CF_risk','Cyc_risk','EQ_risk','FU_risk','PU_risk'],[ast.literal_eval]*len(['CF_risk','Cyc_risk','EQ_risk','FU_risk','PU_risk']))),index_col=[0])

tot_bridge_rail.IncomeGroup = tot_bridge_rail.IncomeGroup.apply(lambda x :x.upper())

country_road_bridge = tot_bridge_road.loc[tot_bridge_road.road_type.isin(['primary','secondary','tertiary'])].groupby(
    ['country'])['CF_risk','Cyc_risk','EQ_risk','FU_risk','PU_risk'].agg(sum_tuples)

country_rail_bridge = tot_bridge_rail.groupby(['country'])['CF_risk','Cyc_risk','EQ_risk','FU_risk','PU_risk'].agg(sum_tuples)
                          

### Step 6: Add additional information to the output datasets

In [ ]:
tot_road_FU = tot_road_FU.merge(incomegroups,left_on='country',right_on='CountryCode').merge(global_countries[['ISO_3digit','wbregion']],left_on='country',right_on='ISO_3digit')
tot_road_PU = tot_road_PU.merge(incomegroups,left_on='country',right_on='CountryCode').merge(global_countries[['ISO_3digit','wbregion']],left_on='country',right_on='ISO_3digit')
tot_road_Cyc = tot_road_Cyc.merge(incomegroups,left_on='country',right_on='CountryCode')
tot_road_EQ = tot_road_EQ.merge(incomegroups,left_on='country',right_on='CountryCode')
tot_road_CF = tot_road_CF.merge(incomegroups,left_on='country',right_on='CountryCode')

tot_rail_FU = tot_rail_FU.merge(incomegroups,left_on='country',right_on='CountryCode').merge(global_countries[['ISO_3digit','wbregion']],left_on='country',right_on='ISO_3digit')
tot_rail_PU = tot_rail_PU.merge(incomegroups,left_on='country',right_on='CountryCode').merge(global_countries[['ISO_3digit','wbregion']],left_on='country',right_on='ISO_3digit')
tot_rail_Cyc = tot_rail_Cyc.merge(incomegroups,left_on='country',right_on='CountryCode')
tot_rail_EQ = tot_rail_EQ.merge(incomegroups,left_on='country',right_on='CountryCode')
tot_rail_CF = tot_rail_CF.merge(incomegroups,left_on='country',right_on='CountryCode')

In [ ]:
%%time
tqdm.pandas()
tot_road_FU = tot_road_FU.progress_apply(lambda x : set_prot_standard(x,prot_lookup,events_FU),axis=1)
tot_road_CF = tot_road_CF.progress_apply(lambda x : set_prot_standard(x,prot_lookup,events_CF),axis=1)

tot_rail_FU = tot_rail_FU.progress_apply(lambda x : set_prot_standard(x,prot_lookup,events_FU),axis=1)
tot_rail_CF = tot_rail_CF.progress_apply(lambda x : set_prot_standard(x,prot_lookup,events_CF),axis=1)

tot_road_FU = tot_road_FU.progress_apply(lambda x : pluvial_design(x,'FU'),axis=1)
tot_road_CF = tot_road_CF.progress_apply(lambda x : pluvial_design(x,'CF'),axis=1)
tot_road_PU = tot_road_PU.progress_apply(lambda x : pluvial_design(x,'PU'),axis=1)
tot_rail_FU = tot_rail_FU.progress_apply(lambda x : pluvial_design_rail(x,'FU'),axis=1)
tot_rail_CF = tot_rail_CF.progress_apply(lambda x : pluvial_design_rail(x,'CF'),axis=1)
tot_rail_PU = tot_rail_PU.progress_apply(lambda x : pluvial_design_rail(x,'PU'),axis=1)

### Step 7: Group datasets to country level

In [ ]:
%%time
FU_country_stats = tot_road_FU.loc[tot_road_FU.road_type.isin(['primary','secondary','tertiary'])]
FU_country_stats = FU_country_stats.groupby(['country'])[events_FU].agg(sum_tuples)

PU_country_stats = tot_road_PU.loc[tot_road_PU.road_type.isin(['primary','secondary','tertiary'])]
PU_country_stats = PU_country_stats.groupby(['country'])[events_PU].agg(sum_tuples)

CF_country_stats = tot_road_CF.loc[tot_road_CF.road_type.isin(['primary','secondary','tertiary'])]
CF_country_stats = CF_country_stats.groupby(['country'])[events_CF].agg(sum_tuples)

EQ_country_stats = tot_road_EQ.loc[tot_road_EQ.road_type.isin(['primary','secondary','tertiary'])]
EQ_country_stats = EQ_country_stats.groupby(['country'])[events_EQ].agg(sum_tuples)

Cyc_country_stats = tot_road_Cyc.loc[tot_road_Cyc.road_type.isin(['primary','secondary','tertiary'])]
Cyc_country_stats = Cyc_country_stats.groupby(['country'])[events_Cyc].agg(sum_tuples)

In [ ]:
FU_country_stats_rail = tot_rail_FU.loc[~(tot_rail_FU.infra_type.isin(rail_to_remove))].groupby(
    ['country'])[events_FU].agg(sum_tuples)
PU_country_stats_rail = tot_rail_PU.loc[~(tot_rail_PU.infra_type.isin(rail_to_remove))].groupby(
    ['country'])[events_PU].agg(sum_tuples)
Cyc_country_stats_rail = tot_rail_Cyc.loc[~(tot_rail_Cyc.infra_type.isin(rail_to_remove))].groupby(
    ['country'])[events_Cyc].agg(sum_tuples)
EQ_country_stats_rail = tot_rail_EQ.loc[~(tot_rail_EQ.infra_type.isin(rail_to_remove))].groupby(
    ['country'])[events_EQ].agg(sum_tuples)
CF_country_stats_rail = tot_rail_CF.loc[~(tot_rail_CF.infra_type.isin(rail_to_remove))].groupby(
    ['country'])[events_CF].agg(sum_tuples)

In [ ]:
%%time
tqdm.pandas()
RPS = [1/5,1/10,1/20,1/50,1/75,1/100,1/200,1/250,1/500,1/1000]
country_risk_PU_road = pd.DataFrame(PU_country_stats.progress_apply(lambda x: calc_risk_total(x,'PU',RPS,events_PU),axis=1).tolist(),index=PU_country_stats.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])
country_risk_FU_road = pd.DataFrame(FU_country_stats.progress_apply(lambda x: calc_risk_total(x,'FU',RPS,events_FU),axis=1).tolist(),index=FU_country_stats.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])
country_risk_PU_rail = pd.DataFrame(PU_country_stats_rail.progress_apply(lambda x: calc_risk_total(x,'PU',RPS,events_PU),axis=1).tolist(),
                                    index=PU_country_stats_rail.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])
country_risk_FU_rail = pd.DataFrame(FU_country_stats_rail.progress_apply(lambda x: calc_risk_total(x,'FU',RPS,events_FU),axis=1).tolist(),
                                    index=FU_country_stats_rail.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])

RPS = [1/10,1/20,1/50,1/100,1/200,1/500,1/1000]
country_risk_CF_road = pd.DataFrame(CF_country_stats.progress_apply(lambda x: calc_risk_total(x,'CF',RPS,events_CF),axis=1).tolist(),
                       index=CF_country_stats.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])
country_risk_CF_rail = pd.DataFrame(CF_country_stats_rail.progress_apply(lambda x: calc_risk_total(x,'CF',RPS,events_CF),axis=1).tolist(),
                       index=CF_country_stats_rail.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])

RPS = [1/50,1/100,1/250,1/500,1/1000]
country_risk_EQ_road = pd.DataFrame(EQ_country_stats.progress_apply(lambda x: calc_risk_total(x,'EQ',RPS,events_EQ),axis=1).tolist(),
                       index=EQ_country_stats.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])
country_risk_EQ_rail = pd.DataFrame(EQ_country_stats_rail.progress_apply(lambda x: calc_risk_total(x,'EQ',RPS,events_EQ),axis=1).tolist(),
                       index=EQ_country_stats_rail.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])

RPS = [1/250,1/475,1/975,1/1500,1/2475]
country_risk_Cyc_road = pd.DataFrame(Cyc_country_stats.progress_apply(lambda x: calc_risk_total(x,'Cyc',RPS,events_Cyc),axis=1).tolist(),
                       index=Cyc_country_stats.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])
country_risk_Cyc_rail = pd.DataFrame(Cyc_country_stats_rail.progress_apply(lambda x: calc_risk_total(x,'Cyc',RPS,events_Cyc),axis=1).tolist(),
                       index=Cyc_country_stats_rail.index,
     columns=['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100'])

### Step 8: Load additional road and railway data

In [ ]:
tot_roads = pd.read_feather(os.path.join(data_path,'summarized','all_road_stats.ft'))
tot_roads = tot_roads.loc[tot_roads.road_type.isin(['primary','secondary','tertiary'])]
tot_roads = tot_roads.merge(global_countries[['ISO_3digit','wbincomena']],left_on='country',right_on='ISO_3digit')
tot_roads = tot_roads.merge(incomegroups,left_on='country',right_on='CountryCode')
tot_len_road = pd.DataFrame(tot_roads.groupby('country')['length'].sum(),columns=['length'])
lookup_length_road = dict(zip(tot_len_road.index,tot_len_road.length))

In [ ]:
tot_rail = pd.read_feather(os.path.join(data_path,'summarized','all_railway_stats.ft'))
tot_rail = tot_rail.loc[~(tot_rail.infra_type.isin(rail_to_remove))]

tot_rail = tot_rail.merge(global_countries[['ISO_3digit','wbincomena']],left_on='country',right_on='ISO_3digit')
tot_rail = tot_rail.merge(incomegroups,left_on='country',right_on='CountryCode')
tot_len_rail = pd.DataFrame(tot_rail.groupby('country')['length'].sum(),columns=['length'])

tot_len = tot_len_road.add(tot_len_rail, fill_value=0)
lookup_length = dict(zip(tot_len.index,tot_len.length))

### Step 9: Calculate the total risk for each country

In [ ]:
country_br_ro = (pd.DataFrame(country_road_bridge['Cyc_risk'].apply(pd.Series)).add(
pd.DataFrame(country_road_bridge['EQ_risk'].apply(pd.Series)), fill_value=0).add(
pd.DataFrame(country_road_bridge['PU_risk'].apply(pd.Series)), fill_value=0).add(
pd.DataFrame(country_road_bridge['FU_risk'].apply(pd.Series)), fill_value=0).add(
pd.DataFrame(country_road_bridge['CF_risk'].apply(pd.Series)), fill_value=0))

country_br_ro.columns = ['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100']

country_br_ra = (pd.DataFrame(country_rail_bridge['Cyc_risk'].apply(pd.Series)).add(
pd.DataFrame(country_rail_bridge['EQ_risk'].apply(pd.Series)), fill_value=0).add(
pd.DataFrame(country_rail_bridge['PU_risk'].apply(pd.Series)), fill_value=0).add(
pd.DataFrame(country_rail_bridge['FU_risk'].apply(pd.Series)), fill_value=0).add(
pd.DataFrame(country_rail_bridge['CF_risk'].apply(pd.Series)), fill_value=0))

country_br_ra.columns = ['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100']

In [ ]:
gdp_country = pd.read_csv(os.path.join(data_path,'input_data','gdp_country.csv'),encoding='iso-8859-1').fillna(0)
GDP_lookup = dict(zip(gdp_country['Country Code'],gdp_country.max(axis=1)))
GDP_lookup_2 = dict(zip(global_countries['GID_0'],global_countries.GDP))

country_risk = country_risk_PU_road.add(country_risk_FU_road, fill_value=0).add(
    country_risk_CF_road, fill_value=0).add(country_risk_EQ_road, fill_value=0).add(country_risk_Cyc_road, fill_value=0).add(
    country_risk_FU_rail, fill_value=0).add(country_risk_CF_rail, fill_value=0).add(country_risk_EQ_rail, fill_value=0).add(
    country_risk_PU_rail, fill_value=0).add(country_risk_Cyc_rail, fill_value=0).add(country_br_ro, fill_value=0).add(
    country_br_ra, fill_value=0)

country_risk = country_risk.loc[country_risk.perc_50 != 0]

country_risk = country_risk.reset_index()
country_risk['GDP'] = country_risk.country.apply(lambda x: gdp_lookup(x))
country_risk['length'] = country_risk.country.apply(lambda x: lookup_length[x])

country_risk = country_risk.loc[country_risk.perc_50 != 0]
country_risk = country_risk.loc[country_risk.GDP != 0]
country_risk.dropna(inplace=True)

### Step 10: Create Figure 5

In [ ]:
total_infra_value = pd.read_csv(os.path.join(data_path,'summarized','total_infrastructure_values.csv'))
country_infra_value = total_infra_value.groupby(['country']).sum()

In [ ]:
name_lookup = dict(zip(global_countries['GID_0'],global_countries['NAME_0']))
inc_lookup = dict(zip(incomegroups['CountryCode'],incomegroups['GroupCode'].apply(lambda x: x.upper())))

color_scheme = ['#F3FFBD','#B2DBBF','#70C1B3','#247BA0'] #['#bae4bc','#7bccc4','#43a2ca','#0868ac']
income_dict = ['LIC', 'LMC', 'UMC', 'HIC']
color_lookup = dict(zip(income_dict,color_scheme))

In [ ]:
country_risk['wbinc'] = country_risk['country'].apply(lambda x : inc_lookup[x])

In [ ]:
fig, axes = plt.subplots(1, 4,figsize=(26,10))  
fig.subplots_adjust(wspace=1)
letters = ['A','B','C','D']

for iter_,ax in enumerate(axes.flat):
    if iter_ == 0:
        top20 = country_risk.sort_values('perc_50',
                                 ascending=False)[['country','perc_0','perc_20','perc_40',
                                                   'perc_50','perc_60','perc_80','perc_100']][:20].groupby('country').sum().sort_values('perc_50',ascending=False)
        top20 = pd.DataFrame(pd.DataFrame(top20.unstack(),columns=['Value']).reset_index(),columns=['level_0','country','Value'])

        top20['Value'] = top20['Value']/1e9
        top20['wbinc'] = top20['country'].apply(lambda x : inc_lookup[x])
        top20['color'] = top20['wbinc'].apply(lambda x : color_lookup[x])
        top20['country'] = top20['country'].apply(lambda x : name_lookup[x])
        sns.boxplot(x=top20.Value,y=top20.country, showfliers=False,ax=ax,linewidth = 0.7)

        ax.xaxis.set_ticks(np.arange(0,20.1,4))
        ax.set_xlabel(xlabel='EAD in Billion USD',fontweight="bold",color='black',fontsize=22) 
        ax.set_ylabel(ylabel='Country',fontweight="bold",color='black',fontsize=25) #
        ax.set_facecolor('#FAF9F9')
        ax.spines['left'].set_color('black')
        ax.spines['bottom'].set_color('black')        
        ax.set_xscale("log", nonposx='clip')
        ax.set_xlim(right=20)
        ax.xaxis.set_major_formatter(ScalarFormatter())
        
        for y in range(20):
            ax.findobj(matplotlib.patches.Patch)[y].set_facecolor(top20.iloc[y]['color'])
            ax.findobj(matplotlib.patches.Patch)[y].set_edgecolor('black')
            
    
    elif iter_ == 1:
        country_risk_gdp = country_risk.copy()
        country_risk_gdp[['perc_0','perc_20','perc_40',
                          'perc_50','perc_60','perc_80','perc_100']] = country_risk_gdp[['perc_0',
                                                                                         'perc_20','perc_40','perc_50','perc_60','perc_80','perc_100']].div(country_risk.GDP, axis=0)

        top20 = country_risk_gdp.sort_values('perc_50',
                                 ascending=False)[['country','perc_0','perc_20','perc_40',
                                                   'perc_50','perc_60','perc_80','perc_100']][:20].groupby('country').sum().sort_values('perc_50',ascending=False)
        top20 = pd.DataFrame(pd.DataFrame(top20.unstack(),columns=['Value']).reset_index(),columns=['level_0','country','Value'])

        top20['Value'] = top20['Value']*100
        top20['wbinc'] = top20['country'].apply(lambda x : inc_lookup[x])
        top20['color'] = top20['wbinc'].apply(lambda x : color_lookup[x])
        top20['country'] = top20['country'].apply(lambda x : name_lookup[x])

        sns.boxplot(x=top20.Value,y=top20.country, showfliers=False,ax=ax,linewidth = 0.7)

        ax.xaxis.set_ticks(np.arange(0,1.1,0.2))
        ax.set_xlabel(xlabel='EAD in % GDP',fontweight="bold",color='black',fontsize=22) 
        ax.set_ylabel(ylabel='') #
        ax.set_facecolor('#FAF9F9')
        ax.spines['left'].set_color('black')
        ax.spines['bottom'].set_color('black')
        for y in range(20):
            ax.findobj(matplotlib.patches.Patch)[y].set_facecolor(top20.iloc[y]['color'])
            ax.findobj(matplotlib.patches.Patch)[y].set_edgecolor('black')
    elif iter_ == 2:

        country_risk_val = country_risk.copy()
        #country_risk_val = country_risk_val.loc[country_risk_val.country != 'MHL']
        country_risk_val = country_risk_val.groupby('country').sum()*100
        country_risk_val[['perc_0','perc_20','perc_40',
                          'perc_50','perc_60','perc_80','perc_100']] = country_risk_val[['perc_0',
                                                                                         'perc_20','perc_40','perc_50','perc_60','perc_80','perc_100']].div(country_infra_value.infra_value, axis=0)
        country_risk_val.reset_index(inplace=True)
        top20 = country_risk_val.sort_values('perc_50',
                                 ascending=False)[['country','perc_0','perc_20','perc_40',
                                                   'perc_50','perc_60','perc_80','perc_100']][0:20].groupby('country').sum().sort_values('perc_50',ascending=False)
        top20 = pd.DataFrame(pd.DataFrame(top20.unstack(),columns=['Value']).reset_index(),columns=['level_0','country','Value'])

        top20['Value'] = top20['Value']
        top20['wbinc'] = top20['country'].apply(lambda x : inc_lookup[x])
        top20['color'] = top20['wbinc'].apply(lambda x : color_lookup[x])
        top20['country'] = top20['country'].apply(lambda x : name_lookup[x])

        sns.boxplot(x=top20.Value,y=top20.country, showfliers=False,ax=ax,linewidth = 0.7)
        ax.set_facecolor('#FAF9F9')

        ax.set_xlabel(xlabel='EAD in %\nInfrastructure value',fontweight="bold",color='black',fontsize=22) 
        ax.set_ylabel(ylabel='') #
        ax.xaxis.set_ticks(np.arange(0,1.1,0.2))

        ax.spines['left'].set_color('black')
        ax.spines['bottom'].set_color('black')
        for y in range(20):
            ax.findobj(matplotlib.patches.Patch)[y].set_facecolor(top20.iloc[y]['color'])
            ax.findobj(matplotlib.patches.Patch)[y].set_edgecolor('black')
        
    elif iter_ == 3:
        country_risk_len = country_risk.copy()
        country_risk_len[['perc_0','perc_20','perc_40',
                          'perc_50','perc_60','perc_80','perc_100']] = country_risk_len[['perc_0',
                                                                                         'perc_20','perc_40','perc_50','perc_60','perc_80','perc_100']].div(country_risk_len.length, axis=0)

        country_risk_len.reset_index(inplace=True)
        top20 = country_risk_len.sort_values('perc_50',
                                 ascending=False)[['country','perc_0','perc_20','perc_40',
                                                   'perc_50','perc_60','perc_80','perc_100']][:20].groupby('country').sum().sort_values('perc_50',ascending=False)
        top20 = pd.DataFrame(pd.DataFrame(top20.unstack(),columns=['Value']).reset_index(),columns=['level_0','country','Value'])

        top20['Value'] = top20['Value']
        top20['wbinc'] = top20['country'].apply(lambda x : inc_lookup[x])
        top20['color'] = top20['wbinc'].apply(lambda x : color_lookup[x])
        top20['country'] = top20['country'].apply(lambda x : name_lookup[x])

        sns.boxplot(x=top20.Value,y=top20.country, showfliers=False,ax=ax,linewidth = 0.7)
        ax.set_facecolor('#FAF9F9')

        ax.set_xlabel(xlabel='EAD in USD/km',fontweight="bold",color='black',fontsize=22) 
        ax.set_ylabel(ylabel='') #
        ax.xaxis.set_ticks(np.arange(0,6001,1500))

        ax.spines['left'].set_color('black')
        ax.spines['bottom'].set_color('black')
        for y in range(20):
            ax.findobj(matplotlib.patches.Patch)[y].set_facecolor(top20.iloc[y]['color'])
            ax.findobj(matplotlib.patches.Patch)[y].set_edgecolor('black')

    ax.text(-1., 0.99, '{})'.format(letters[iter_]), transform=ax.transAxes,fontweight="bold",color='black', fontsize=18,
        verticalalignment='top', bbox= dict(boxstyle='round', facecolor='white', alpha=0.5,linewidth=0))

    ax.tick_params(axis = 'both',labelcolor='black',color='black',labelsize=20) #

fig.tight_layout()

fig.savefig(os.path.join(figure_path,'Fig5_losses_ranked.png'),dpi=450)

### Step 11: Creation of csv file with total country risk per hazard

In [ ]:
EQ_bridge_road = pd.DataFrame(country_road_bridge['EQ_risk'].apply(pd.Series))
EQ_bridge_road.columns = ['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100']
EQ_bridge_rail = pd.DataFrame(country_rail_bridge['EQ_risk'].apply(pd.Series))
EQ_bridge_rail.columns = ['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100']
EQ = pd.DataFrame(country_risk_EQ_road.add(country_risk_EQ_rail, fill_value=0).add(
    EQ_bridge_road, fill_value=0).add(EQ_bridge_rail, fill_value=0))
EQ.columns = pd.MultiIndex.from_product([['EQ'],EQ.columns])

In [ ]:
FU_bridge_road = pd.DataFrame(country_road_bridge['FU_risk'].apply(pd.Series))
FU_bridge_road.columns = ['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100']
FU_bridge_rail = pd.DataFrame(country_rail_bridge['FU_risk'].apply(pd.Series))
FU_bridge_rail.columns = ['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100']
FU = pd.DataFrame(country_risk_FU_road.add(country_risk_FU_rail, fill_value=0).add(FU_bridge_road, fill_value=0).add(
    FU_bridge_rail, fill_value=0))
FU.columns = pd.MultiIndex.from_product([['FU'],FU.columns])

In [ ]:
PU_bridge_road = pd.DataFrame(country_road_bridge['PU_risk'].apply(pd.Series))
PU_bridge_road.columns = ['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100']
PU_bridge_rail = pd.DataFrame(country_rail_bridge['PU_risk'].apply(pd.Series))
PU_bridge_rail.columns = ['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100']
PU = pd.DataFrame(country_risk_PU_road.add(country_risk_PU_rail, fill_value=0).add(PU_bridge_road, fill_value=0).add(
    PU_bridge_rail, fill_value=0))
PU.columns = pd.MultiIndex.from_product([['PU'],PU.columns])

In [ ]:
CF_bridge_road = pd.DataFrame(country_road_bridge['CF_risk'].apply(pd.Series))
CF_bridge_road.columns = ['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100']
CF_bridge_rail = pd.DataFrame(country_rail_bridge['CF_risk'].apply(pd.Series))
CF_bridge_rail.columns = ['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100']
CF = pd.DataFrame(country_risk_CF_road.add(country_risk_CF_rail, fill_value=0).add(CF_bridge_road, fill_value=0).add(
    CF_bridge_rail, fill_value=0))
CF.columns = pd.MultiIndex.from_product([['CF'],CF.columns])

In [ ]:
Cyc_bridge_road = pd.DataFrame(country_road_bridge['Cyc_risk'].apply(pd.Series))
Cyc_bridge_road.columns = ['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100']
Cyc_bridge_rail = pd.DataFrame(country_rail_bridge['Cyc_risk'].apply(pd.Series))
Cyc_bridge_rail.columns = ['perc_0','perc_20','perc_40','perc_50','perc_60','perc_80','perc_100']
Cyc = pd.DataFrame(country_risk_Cyc_road.add(country_risk_Cyc_rail, fill_value=0).add(Cyc_bridge_road, fill_value=0).add(
    Cyc_bridge_rail, fill_value=0))
Cyc.columns = pd.MultiIndex.from_product([['Cyc'],Cyc.columns])

In [ ]:
country_all_hazards = pd.concat([EQ,FU,PU,CF,Cyc],sort=True,axis=1)

In [ ]:
mh_risk = country_all_hazards.sum(level=1,axis=1)
mh_risk.columns = pd.MultiIndex.from_product([['MH'],mh_risk.columns])
country_all_hazards = country_all_hazards.merge(mh_risk,left_index=True,right_index=True).fillna(0)

In [ ]:
country_all_hazards[('desc','full_name')] = country_all_hazards.index.to_series().apply(lambda x : name_lookup[x])
country_all_hazards[('desc','wbinc')] = country_all_hazards.index.to_series().apply(lambda x : inc_lookup[x])
country_all_hazards[('desc','GDP')] = country_all_hazards.index.to_series().apply(lambda x: gdp_lookup(x))
country_all_hazards[('desc','length')] = country_all_hazards.index.to_series().apply(lambda x: lookup_length[x])
country_infra_value.columns = pd.MultiIndex.from_product([['desc'],['infra_value']])
country_all_hazards = country_all_hazards.merge(country_infra_value,left_index=True,right_index=True)

In [ ]:
country_all_hazards.reindex(['desc','MH','EQ','FU','PU','CF','Cyc'],axis=1,level=0).to_csv(os.path.join(data_path,'output_data','EAD_countries.csv'))